In [1]:
import os, sys
import numpy as np
import matplotlib.pyplot as plt
import mask_rcnn
import mask_rcnn_utils
import eye_segmentation
import cv2
from tqdm import tqdm

import tensorflow as tf
import keras
print(keras.__version__)
import keras.backend as K
import keras.layers as KL
import keras.engine as KE
import keras.models as KM
from tensorflow.keras.preprocessing.image import load_img, img_to_array

Using TensorFlow backend.


STEPS PER EPOCH:  2972
VALIDATION STEPS:  801
2.3.1


In [ ]:
# segmentation dataset
DATASET_DIR = '/home/naproxa/cs271proj/Semantic_Segmentation_Dataset' # replace with your path
TRAIN_IMAGE_DIR = DATASET_DIR + '/train/images'
TRAIN_LABELS_DIR = DATASET_DIR + '/train/labels'
VAL_IMAGE_DIR = DATASET_DIR + '/validation/images'
VAL_LABELS_DIR = DATASET_DIR + '/validation/labels'
TEST_IMAGE_DIR = DATASET_DIR + '/test/images'

print(len(os.listdir(TRAIN_IMAGE_DIR)))
print(len(os.listdir(VAL_IMAGE_DIR)))

# image_ids = next(os.walk(TEST_IMAGE_DIR))[2]
# for image_id in image_ids:
#     print(image_id[:image_id.index('.')])

In [ ]:
assert (len(os.listdir(TRAIN_IMAGE_DIR)) == len(os.listdir(TRAIN_LABELS_DIR))) # 8916
np.set_printoptions(threshold=sys.maxsize)

images = sorted(os.listdir(TRAIN_IMAGE_DIR))
labels = sorted(os.listdir(TRAIN_LABELS_DIR))
print(len(images))

# count = 0
for i in range(len(images)):
    print(labels[i])
#     image = load_img(TRAIN_IMAGE_DIR+'/'+images[i])
    
    label = np.load(TRAIN_LABELS_DIR+'/'+labels[i])
    print(label)
#     plt.figure()
#     plt.imshow(image)
#     plt.savefig('/home/jjonathanmak/cs271proj/image1.png')
#     plt.figure()
    
#     plt.imshow(image)
#     plt.imshow(label, alpha=0.3)
    
#     plt.figure()
 
#     plt.imshow(label)
#     plt.savefig('/home/jjonathanmak/cs271proj/label1.png')
    break
#     count += 1
#     if count == 3: break

In [ ]:
def load_data(IMAGE_DIR, LABEL_DIR):

    images = []
    image_list = sorted(os.listdir(IMAGE_DIR))
    if LABEL_DIR:
        labels = []
        label_list = sorted(os.listdir(LABEL_DIR))

    for file in tqdm(range(len(image_list))):

        image_path = IMAGE_DIR +'/'+image_list[i]
        
        # image is duplicated across channels, take 1st channel
        img = img_to_array(load_img(image_path), dtype=np.uint8)
         
        images.append(img[:, :, 0])
        
        if LABEL_DIR:
            label_path = LABEL_DIR +'/'+label_list[i]
            label = np.load(label_path)
            labels.append(label)

    if LABEL_DIR:
        return np.array(images), np.array(labels)
    else:
        return np.array(images)

train_images, train_labels = load_data(TRAIN_IMAGE_DIR, TRAIN_LABELS_DIR)
val_images, val_labels = load_data(VAL_IMAGE_DIR, VAL_LABELS_DIR)
test_images = load_data(TEST_IMAGE_DIR, None)

print('Train images: ', train_images.shape)
print('Train labels: ', train_labels.shape)
print('Val images: ', val_images.shape)
print('Val labels: ', val_labels.shape)
print('Test images: ', test_images.shape)

In [ ]:
BATCH_SIZE = 8

train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).batch(BATCH_SIZE)
val_dataset = tf.data.Dataset.from_tensor_slices((val_images, val_labels)).batch(BATCH_SIZE)

In [ ]:
# Train
model = eye_segmentation.train_mrcnn(load_last=False)